In [ ]:
import torch  
import torch.nn.functional as F 
import numpy as np
import math
import cv2
import time

def gaussian(window_size, sigma):
    gauss =  torch.Tensor([math.exp(-(x - window_size//2)**2/float(2*sigma**2)) for x in range(window_size)])
    return gauss/gauss.sum()

def cr_window(window_size, channel=1):
    conv1 = gaussian(window_size=window_size, sigma=1.5).unsqueeze(1)
    conv2 = conv1.mm(conv1.t()).float().unsqueeze(0).unsqueeze(0)

    window = torch.Tensor(conv2.expand(channel, 1, window_size, window_size).contiguous())

    return window

def ssim(img1, img2, window_size=11, val_range=255, window=None, size_average=True, full=False):   
    L = val_range
    
    try:
        _, channels, height, width = img1.size()
    except:
        channels, height, width = img1.size()

    if window is None:
        real_size = min(window_size, height, width) 
        window = cr_window(real_size, channel=channels).to(img1.device)
    
    pad = window_size//2
    mu1 = F.conv2d(img1, window, padding=pad, groups=channels)
    mu2 = F.conv2d(img2, window, padding=pad, groups=channels)
    
    mu1_sq = mu1 ** 2
    mu2_sq = mu2 ** 2 
    mu12 = mu1 * mu2

    sigma1_sq = F.conv2d(img1 * img1, window, padding=pad, groups=channels) - mu1_sq
    sigma2_sq = F.conv2d(img2 * img2, window, padding=pad, groups=channels) - mu2_sq
    sigma12 =  F.conv2d(img1 * img2, window, padding=pad, groups=channels) - mu12

    C1 = (0.01 ) ** 2 
    C2 = (0.03 ) ** 2 

    contrast_metric = (2.0 * sigma12 + C2) / (sigma1_sq + sigma2_sq + C2)
    contrast_metric = torch.mean(contrast_metric)

    numerator1 = 2 * mu12 + C1  
    numerator2 = 2 * sigma12 + C2
    denominator1 = mu1_sq + mu2_sq + C1 
    denominator2 = sigma1_sq + sigma2_sq + C2

    ssim_score = (numerator1 * numerator2) / (denominator1 * denominator2)

    if size_average:
        ret = ssim_score.mean() 
    else: 
        ret = ssim_score.mean(1).mean(1).mean(1)
    
    if full:
        return ret, contrast_metric
    
    return ret

In [ ]:
gauss_dis = gaussian(11, 1.5)
window = cr_window(11, 3)

load_images = lambda path, h, w: cv2.resize(cv2.cvtColor(cv2.imread(path, cv2.IMREAD_UNCHANGED), cv2.COLOR_BGR2RGB), ((w, h)))
tensorify = lambda x: torch.Tensor(x.transpose((2, 0, 1))).unsqueeze(0).float().div(255.0)

In [ ]:
Distortion_image_1 = ['/content/i01_16_1.bmp', '/content/i01_16_2.bmp', '/content/i01_16_3.bmp', '/content/i01_16_4.bmp', '/content/i01_16_5.bmp']
time_mean_list = []

for image in Distortion_image_1:
  start = time.time()

  img_tensor = tensorify(load_images("/content/I01.bmp", 400, 300))
  noise_tensor = tensorify(load_images(image, 400, 300))

  test = ssim(img_tensor, noise_tensor)
  result = test.detach().cpu().numpy()
  print("image: ", image)
  print("score: %.4f" % result.item())
  print("time: ", time.time() - start)
  time_data = time.time() - start
  time_mean_list.append(time_data)
  print("")

print("time_mean: ", np.mean(time_mean_list))

image:  /content/i01_16_1.bmp
score: 0.9955
time:  0.9464876651763916

image:  /content/i01_16_2.bmp
score: 0.9919
time:  0.8976011276245117

image:  /content/i01_16_3.bmp
score: 0.9427
time:  0.8980822563171387

image:  /content/i01_16_4.bmp
score: 0.9521
time:  0.8998081684112549

image:  /content/i01_16_5.bmp
score: 0.8078
time:  0.9128344058990479

time_mean:  0.9119059085845947


In [ ]:
Distortion_image_2 = ['/content/i02_16_1.bmp', '/content/i02_16_2.bmp', '/content/i02_16_3.bmp', '/content/i02_16_4.bmp', '/content/i02_16_5.bmp']
time_mean_list = []

for image in Distortion_image_2:
  start = time.time()

  img_tensor = tensorify(load_images("/content/I02.bmp", 400, 300))
  noise_tensor = tensorify(load_images(image, 400, 300))

  test = ssim(img_tensor, noise_tensor)
  result = test.detach().cpu().numpy()
  print("image: ", image)
  print("score: %.4f" % result.item())
  print("time: ", time.time() - start)
  time_data = time.time() - start
  time_mean_list.append(time_data)
  print("")

print("time_mean: ", np.mean(time_mean_list))

image:  /content/i02_16_1.bmp
score: 0.9827
time:  0.9720029830932617

image:  /content/i02_16_2.bmp
score: 0.9750
time:  0.8892922401428223

image:  /content/i02_16_3.bmp
score: 0.7900
time:  0.9191651344299316

image:  /content/i02_16_4.bmp
score: 0.8809
time:  0.9287288188934326

image:  /content/i02_16_5.bmp
score: 0.4991
time:  0.903350830078125

time_mean:  0.9234544277191162


In [ ]:
Distortion_image_3 = ['/content/i03_11_1.BMP', '/content/i03_11_2.bmp', '/content/i03_11_3.bmp', '/content/i03_11_4.bmp', '/content/i03_11_5.bmp', '/content/i03_16_1.bmp', '/content/i03_16_2.bmp', '/content/i03_16_3.bmp', '/content/i03_16_4.bmp', '/content/i03_16_5.bmp']
time_mean_list = []

for image in Distortion_image_3:
  start = time.time()

  img_tensor = tensorify(load_images("/content/I03.bmp", 400, 300))
  noise_tensor = tensorify(load_images(image, 400, 300))

  test = ssim(img_tensor, noise_tensor)
  result = test.detach().cpu().numpy()
  print("image: ", image)
  print("score: %.4f" % result.item())
  print("time: ", time.time() - start)
  time_data = time.time() - start
  time_mean_list.append(time_data)
  print("")

print("time_mean: ", np.mean(time_mean_list))

image:  /content/i03_11_1.BMP
score: 0.9350
time:  0.9411866664886475

image:  /content/i03_11_2.bmp
score: 0.8715
time:  0.8875589370727539

image:  /content/i03_11_3.bmp
score: 0.7960
time:  0.9303674697875977

image:  /content/i03_11_4.bmp
score: 0.7179
time:  1.1756706237792969

image:  /content/i03_11_5.bmp
score: 0.6786
time:  0.9115631580352783

image:  /content/i03_16_1.bmp
score: 0.9771
time:  0.9038429260253906

image:  /content/i03_16_2.bmp
score: 0.9747
time:  0.9161560535430908

image:  /content/i03_16_3.bmp
score: 0.8660
time:  0.9224298000335693

image:  /content/i03_16_4.bmp
score: 0.9017
time:  0.8965530395507812

image:  /content/i03_16_5.bmp
score: 0.6764
time:  0.9373311996459961

time_mean:  0.9436602115631103


In [ ]:
Distortion_image_4 = ['/content/i04_16_1.bmp', '/content/i04_16_2.bmp', '/content/i04_16_3.bmp', '/content/i04_16_4.bmp', '/content/i04_16_5.bmp']
time_mean_list = []

for image in Distortion_image_4:
  start = time.time()

  img_tensor = tensorify(load_images("/content/I04.bmp", 400, 300))
  noise_tensor = tensorify(load_images(image, 400, 300))

  test = ssim(img_tensor, noise_tensor)
  result = test.detach().cpu().numpy()
  print("image: ", image)
  print("score: %.4f" % result.item())
  print("time: ", time.time() - start)
  time_data = time.time() - start
  time_mean_list.append(time_data)
  print("")

print("time_mean: ", np.mean(time_mean_list))

image:  /content/i04_16_1.bmp
score: 0.9918
time:  0.8907532691955566

image:  /content/i04_16_2.bmp
score: 0.9869
time:  0.9112069606781006

image:  /content/i04_16_3.bmp
score: 0.9015
time:  0.9119734764099121

image:  /content/i04_16_4.bmp
score: 0.9298
time:  0.901188850402832

image:  /content/i04_16_5.bmp
score: 0.6970
time:  0.896723747253418

time_mean:  0.9030497074127197


In [ ]:
Distortion_image_5 = ['/content/i05_16_1.bmp', '/content/i05_16_2.bmp', '/content/i05_16_3.bmp', '/content/i05_16_4.bmp', '/content/i05_16_5.bmp']
time_mean_list = []

for image in Distortion_image_5:
  start = time.time()

  img_tensor = tensorify(load_images("/content/I05.bmp", 400, 300))
  noise_tensor = tensorify(load_images(image, 400, 300))

  test = ssim(img_tensor, noise_tensor)
  result = test.detach().cpu().numpy()
  print("image: ", image)
  print("score: %.4f" % result.item())
  print("time: ", time.time() - start)
  time_data = time.time() - start
  time_mean_list.append(time_data)
  print("")

print("time_mean: ", np.mean(time_mean_list))

image:  /content/i05_16_1.bmp
score: 0.9887
time:  0.9480907917022705

image:  /content/i05_16_2.bmp
score: 0.9823
time:  0.8916668891906738

image:  /content/i05_16_3.bmp
score: 0.8782
time:  0.8921511173248291

image:  /content/i05_16_4.bmp
score: 0.9081
time:  0.9122333526611328

image:  /content/i05_16_5.bmp
score: 0.6563
time:  0.9050157070159912

time_mean:  0.9111969947814942
